In [38]:
import math
import numpy as np
import pandas as pd

from avipy import qty, atmosphere as atm

# Landing Performance Assignment
## Deliberable 2: Actual Landing Distance

Calculate the actual landing distance for a landing weight of 250000 kg at sea level.

Values:
- Final approach speed, $ v_{ref} + 5 Kts $
- Flare time, $ t_f = 5.2 s $
- Flare speed ratio, $ FSR = 0.978 $
- Transition time, $t_{tr} = 1.8 s$
- Transition speed decay, $decay = 0.982 $
- Thrust at touchdown per engine, $ T_{td} = 3400 lbs $

Assumptions:
- No reverse thrust, no slope, no wind
- 1 second to apply maximum braking power
- Ground idle reverse thrust as follows:
- Any flare height is negligible, thus  $TAS_{stall} = IAS_{stall} = 122 Kts$
- Stall speed to ref speed, $ v_{ref} = v_{stall} \cdot 1.23 $
- Stall speed to touchdown speed, $ v_{td} = v_{stall} \cdot 1.18 $


| TAS [kts] | Total Idle Forward Thrust [lbs] |
|-----------|---------------------------------|
| 130 | 4896 |
| 120 | 5266 |
| 110 | 5517 |
| 100 | 5949 |
| 80 | 6580 |
| 60 | 7235 |
| 40 | 8157 |
| 20 | 8991 |
| 0 | 9875 |

First, let's define these values in Python. As well as some aircraft related constants from deliverable 1 of the landing assignment. This is viable, as the aircraft between the two assignments is the same.

In [39]:
mass = qty.Mass(250000)
weight = qty.Force.Kg(mass)
v_stall = qty.Velocity.Kts(122)
v_stall_to_ref = 1.23
v_stall_to_td = 1.18
t_flare = qty.Time(5.2)
fsr = 0.978
t_trans = qty.Time(1.8)
speed_decay = 0.982
thrust_td = qty.Force.Kg(qty.Mass.Lbs(3400 * 2))
mu_dry = 0.54
density = 1.2252
cl_gnd = 0.126
cd_gnd = 0.213
surface = qty.Area(436.8)

speed_thrust_segments = [
    (qty.Velocity.Kts(130), qty.Force.Kg(qty.Mass.Lbs(4896))),
    (qty.Velocity.Kts(120), qty.Force.Kg(qty.Mass.Lbs(5266))),
    (qty.Velocity.Kts(110), qty.Force.Kg(qty.Mass.Lbs(5517))),
    (qty.Velocity.Kts(100), qty.Force.Kg(qty.Mass.Lbs(5949))),
    (qty.Velocity.Kts(80), qty.Force.Kg(qty.Mass.Lbs(6580))),
    (qty.Velocity.Kts(60), qty.Force.Kg(qty.Mass.Lbs(7235))),
    (qty.Velocity.Kts(40), qty.Force.Kg(qty.Mass.Lbs(8157))),
    (qty.Velocity.Kts(20), qty.Force.Kg(qty.Mass.Lbs(8991))),
    (qty.Velocity.Kts(0), qty.Force.Kg(qty.Mass.Lbs(9875))),
]

print(v_stall)


62.76 m/s
62.76 m/s


The first thing that needs to be calculated is the value for $ C_{L_{max}} $, since many equations use this value. This is obtained by

$ \large C_{L_{max}} = \frac{2 \cdot W}{\rho \cdot {TAS}_{stall}^2 \cdot S} $

In [40]:
print(v_stall)


cl_max = (2 * 9.81 * 200000) / (1.225 * v_stall**2 * surface)

cl_max

62.76 m/s


1.8617169631927402

In [41]:


def get_tas(weight: qty.Force) -> tuple[qty.Velocity, qty.Velocity, qty.Velocity, qty.Velocity]:
    """
    Returns the true air speed of (Ref Speed, Touchdown Speed, Brakes-on Speed, Stall Speed) in m/s.
    """
    
    ias_stall = math.sqrt((2 * weight) / (surface * cl_max * 1.225))
    v_stall = qty.Velocity(ias_stall * math.sqrt(1.225 / density))
    v_ref = qty.Velocity(v_stall * v_stall_to_ref)
    v_app = qty.Velocity(v_ref + qty.Velocity.Kts(5))
    v_td = qty.Velocity(v_app * fsr)
    v_bo = qty.Velocity(v_td * speed_decay)
    
    return v_stall, v_td, v_bo, v_app, v_ref

In [42]:
v_stall, v_td, v_bo, v_app, v_ref = get_tas(qty.Force.Kg(250000))

print(v_ref, v_td, v_bo, v_stall, v_app)

86.30 m/s 86.92 m/s 85.35 m/s 70.16 m/s 88.87 m/s


Calculate the flare distance, using:\
$ \large S_{air} = v_{flare_{avg}} \cdot t_{flare} $

In [43]:
v_avg = qty.Velocity((v_app + v_td) / 2)
dist_air = qty.Distance(v_avg * t_flare)

print(v_avg, dist_air)

87.90 m/s 457.06 meters


Now, calculate the transition distance. First, the average speed during the transition time can be obtained by taking the average between the touchdown speed and the brakes-on speed.

In [44]:
v_trans_avg = qty.Velocity((v_td + v_bo) / 2)

print(v_trans_avg)

86.14 m/s


Then the transition distance can be obtained by applying the transition speed and transition time.

In [45]:
dist_trans = qty.Distance(v_trans_avg * t_trans)

print(dist_trans)

155.05 meters


The assumption is made that during the 1 second brake delay, the brakes apply linearly. Therefore, the acceleration and speed decay during this 1 second can be obtained by taking the average values for acceleration. 

In [46]:
brake_delay_dist = qty.Distance(1 * v_bo)

lift_bo = 0.5 * density * v_bo**2 * surface * cl_gnd
drag_bo = 0.5 * density * v_bo**2 * surface * cd_gnd

accel_bo = (9.81 * (thrust_td - drag_bo - mu_dry * (weight - lift_bo))) / weight
accel_brake_delay_avg = accel_bo / 2

v_bo_pre_delay = v_bo
v_bo = qty.Velocity(v_bo + accel_brake_delay_avg * 1)

brake_delay_dist = qty.Distance((v_bo_pre_delay + v_bo) / 2)

print(brake_delay_dist, accel_brake_delay_avg)

83.78 meters -3.1533732543411133


The ground run distance can be calculated with a multi-step practical approach:

1. Split the round run up into multiple speed segments. 
2. Calculate the acceleration at the upper speed and lower speed of the segment.
3. Calculate the average speed and acceleration during the segment
4. With the average speed and acceleration, calculate the distance travelled throughout the segment.
5. Add up all segment distances, the total is the approximate ground run distance.

To calculate this in Python, two functions will be defined. The first function `get_brake_dist` takes the brakes-on speed, and divides the ground run into multiple speed segments. For every segment, the second function will be called.

The second function `get_segment_dist` takes the speed bounds of the segment, and calculates the segment distance using the described approach. 

To calculate the acceleration at a given speed. The following equation is used:

$\large a = \frac{g \cdot (T - D - \mu \cdot (W - L))}{W}$

where

$L = \frac{1}{2} \cdot \rho_h \cdot v^2 \cdot S \cdot C_{L_{gnd}}$

and 

$D = \frac{1}{2} \cdot \rho_h \cdot v^2 \cdot S \cdot C_{D_{gnd}}$

In [47]:
def get_brake_dist(v_bo: qty.Velocity, segments: list[tuple[qty.Velocity, qty.Force]]) -> qty.Distance:
    brake_dist = 0
    
    # For every speed - thrust in the list of speed - thrust segments, execute the function defined below with a (speed, thrust) and the next (speed, thrust) in the list

    # For example, the first time, the function will be called with:
    #   Speed range of 85.35 to 66.88 m/s
    #   Thrust range of 30258.22 to 21785.91 N
    #   These values can also be found in the printed table a couple cells below.
    for i in range(len(segments)):
        if i >= len(segments) - 1:
            return qty.Distance(brake_dist)

        segment = get_segment_dist(segments[i], segments[i + 1])
        brake_dist += segment
    
    return None
    
def get_segment_dist(upper: tuple[qty.Velocity, qty.Force], lower: tuple [qty.Velocity, qty.Force]) -> qty.Distance:
    # Get the upper and the lower speed and thrust from the speed, thrust segment table
    speed_upper, speed_lower = upper[0], lower[0]
    thrust_upper, thrust_lower = upper[1], lower[1]
    
    # With the values for upper and lower speed, the upper and lower drag can be DETERMINED
    drag_upper = qty.Force(0.5 * density * speed_upper**2 * surface * cd_gnd)
    drag_lower = qty.Force(0.5 * density * speed_lower**2 * surface * cd_gnd)
    
    # Likewise for the lift
    lift_upper = qty.Force(0.5 * density * speed_upper**2 * surface * cl_gnd)
    lift_lower = qty.Force(0.5 * density * speed_lower**2 * surface * cl_gnd)

    # Then the upper and lower acceleration
    accel_upper = (9.81 *  (thrust_upper - drag_upper - mu_dry * (weight - lift_upper))) / weight
    accel_lower = (9.81 *  (thrust_lower - drag_lower - mu_dry * (weight - lift_lower))) / weight

    # With the upper and lower values for acceleration, 
    # calculate the average and with that average calculate the distance travelled 
    # from the upper speed in the segment to the lower speed in the segment
    accel_avg = (accel_upper + accel_lower) / 2
    v_avg = qty.Velocity((speed_upper + speed_lower) / 2)
    delta_v = qty.Velocity(speed_upper - speed_lower)
    segment_dist = qty.Distance(-(v_avg * delta_v / accel_avg))
    
    return qty.Distance(segment_dist)

Insert the speed and thrust at the brakes-on speed in the beginning of the speed_thrust list. Then obtain the distance using the function defined above.

In [48]:
speed_thrust_segments.insert(0, (v_bo, thrust_td))

dist_brake = get_brake_dist(v_bo, speed_thrust_segments)
print(dist_brake)

593.36 meters


These are the speed / thrust segments:

In [49]:
for speed, thrust in speed_thrust_segments:
    print(speed, thrust)

82.20 m/s 30258.22 Newtons
66.88 m/s 21785.91 Newtons
61.73 m/s 23432.32 Newtons
56.59 m/s 24549.20 Newtons
51.44 m/s 26471.49 Newtons
41.16 m/s 29279.27 Newtons
30.87 m/s 32193.85 Newtons
20.58 m/s 36296.51 Newtons
10.29 m/s 40007.59 Newtons
0.00 m/s 43941.16 Newtons


To calculate the distance, this table with speed / thrust "timestamps" is passed to the function defined above

In [50]:
dist_brake = get_brake_dist(v_bo, speed_thrust_segments)

In [51]:
dist_total = qty.Distance(dist_air + dist_trans + dist_brake + brake_delay_dist)

print(dist_total)

1289.25 meters
